In [7]:
import flask

import numpy as np
import pandas as pd
import pickle
from collections import defaultdict

from sklearn.neighbors import NearestNeighbors


from fuzzywuzzy import process
from fuzzywuzzy import fuzz



In [8]:
with open('./data/df_merged_cleaned.pkl', 'rb') as picklefile:
    df_merged = pickle.load(picklefile)

with open('./data/topic_names.pkl', 'rb') as picklefile:
    topic_names = pickle.load(picklefile)

with open('./data/lda_model.pkl', 'rb') as picklefile:
    lda_mod = pickle.load(picklefile)

with open('./data/vectorizer.pkl', 'rb') as picklefile:
    vect_mod = pickle.load(picklefile)

with open('./data/lda_model_data.pkl', 'rb') as picklefile:
    lda_data = pickle.load(picklefile)

cleaned_talks = df_merged['text']

titles = df_merged['title']

In [9]:
app = flask.Flask(__name__)

In [10]:
def get_recommendations(first_article, model, vectorizer, training_vectors,title, ind):

    new_vec = model.transform(
        vectorizer.transform([first_article]))

    nn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
    nn.fit(training_vectors)
    results = nn.kneighbors(new_vec)
    recommend_list = results[1][0]
    scores = results[0]

    rec_dict = defaultdict(list)

    ss = np.array(scores).flat
    for i, resp in enumerate(recommend_list):

        rec_dict["0"].append(df_merged.iloc[resp,1])
#         rec_dict['title'] = df_merged[title]
    
    rec_dict["0"].append(title)
    rec_dict["0"].append(topic_names.iloc[ind,0])
    return rec_dict

In [11]:
# get_recommendations(cleaned_talks[1], lda_mod, vect_mod,
#                                        lda_data, 'Do schools kill creativity?', 1)

In [12]:
@app.route("/")
def visualize_page():

    
    with open("ted_html.html", 'r') as viz_file:
        return viz_file.read()

In [ ]:
@app.route("/ted", methods=["POST"])
def ted():
    """
    When A POST request with json data is made to this uri,
    Read the example from the json, predict probability and
    send it with a response
    """
    # Get decision score for our example that came with the request
    data = flask.request.json
    print(data)
    X = data#["example"])
    #X = str(X[0])
    title, score, talk_ind = process.extractOne(X, titles, scorer=fuzz.token_set_ratio)
    recs = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)

    print(recs.values())
    # Put the result in a nice dict so we can send it as json
    #results = {"recommends": recs[1]}
    #print(results)
    return flask.jsonify(recs['0'])
    #return flask.json.dump(recs,fp)
#--------- RUN WEB APP SERVER ------------#

# Start the app server on port 80
# (The default website port)
app.run(host='0.0.0.0')
app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2019 15:17:50] "GET / HTTP/1.1" 200 -
/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 15:18:03] "POST /ted HTTP/1.1" 200 -


mapping the brain
dict_values([['Many people think the lines on the map no longer matter, but Parag Khanna says they do. Using maps of the past and present, he explains the root causes of border conflicts worldwide and proposes simple yet cunning solutions for each.', 'Over the past few centuries, Western cultures have been very good at creating general prosperity for themselves. Historian Niall Ferguson asks: Why the West, and less so the rest? He suggests half a dozen big ideas from Western culture -- call them the 6 killer apps -- that promote wealth, stability and innovation. And in this new century, he says, these apps are all shareable.', 'Severine Autesserre studies the Democratic Republic of Congo, which is in the middle of the deadliest conflict since World War II; it\'s been called "the largest ongoing humanitarian crisis in the world.” The conflict seems hopelessly, unsolvably large. But her insight from decades of listening and engaging: The conflicts are often locally base

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 15:18:19] "POST /ted HTTP/1.1" 200 -


brain
dict_values([['Treo creator Jeff Hawkins urges us to take a new look at the brain -- to see it not as a fast processor, but as a memory system that stores and plays back experiences to help us predict, intelligently, what will happen next.', 'Can we use our brains to directly control machines? Miguel Nicolelis suggests yes, showing how a clever monkey in the US learned to control a robot arm in Japan purely with its thoughts. The research has big implications for quadraplegic people -- and in fact, it powered the exoskeleton that kicked off the 2014 World Cup.', 'You may remember neuroscientist Miguel Nicolelis — he built the brain-controlled exoskeleton that allowed a paralyzed man to kick the first ball of the 2014 World Cup. What’s he working on now? Building ways for two minds (rats and monkeys, for now) to send messages brain to brain. Watch to the end for an experiment that, as he says, will go to "the limit of your imagination."', 'As humans, we can perceive less than a te

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 15:19:10] "POST /ted HTTP/1.1" 200 -


new way to make music
dict_values([['Designer Jared Ficklin creates wild visualizations that let us see music, using color and even fire (a first for the TED stage) to analyze how sound makes us feel. He takes a brief digression to analyze the sound of a skatepark -- and how audio can clue us in to developing creativity.', 'Meet the “motion microscope,” a video-processing tool that plays up tiny changes in motion and color impossible to see with the naked eye. Video researcher Michael Rubinstein plays us clip after jaw-dropping clip showing how this tech can track an individual’s pulse and heartbeat simply from a piece of footage. Watch him re-create a conversation by amplifying the movements from sound waves bouncing off a bag of chips. The wow-inspiring and sinister applications of this tech you have to see to believe.', 'Violinist Sirena Huang gives a technically brilliant and emotionally nuanced performance. In a charming interlude, the 11-year-old praises the timeless design of he

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 15:19:40] "POST /ted HTTP/1.1" 200 -


do schools kill creativity
dict_values([['Sir Ken Robinson makes an entertaining and profoundly moving case for creating an education system that nurtures (rather than undermines) creativity.', "With profound simplicity, Coach John Wooden redefines success and urges us all to pursue the best in ourselves. In this inspiring talk he shares the advice he gave his players at UCLA, quotes poetry and remembers his father's wisdom.", 'How can you help kids get a good start? In this heartfelt and personal talk, Colin Powell, the former U.S. Secretary of State, asks parents, friends and relatives to support children, starting before they even get to primary school, through community and a strong sense of responsibility.', "Imagine a country where girls must sneak out to go to school, with deadly consequences if they get caught learning. This was Afghanistan under the Taliban, and traces of that danger remain today. 22-year-old Shabana Basij-Rasikh runs a school for girls in Afghanistan. She cel